In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

db = mysql.connector.connect(host = "localhost",
                             username = "root",
                             password = "150705",
                             database = "ecommerce")
cur = db.cursor()

# Calculate the number of orders per month in 2018.


In [6]:
query = """ select monthname(order_purchase_timestamp) month, count(order_id) order_counts
from orders where year(order_purchase_timestamp) =2018
group by month """


cur.execute(query)

data = cur.fetchall()

data



[('July', 25168),
 ('August', 26048),
 ('February', 26912),
 ('June', 24668),
 ('March', 28844),
 ('January', 29076),
 ('May', 27492),
 ('April', 27756),
 ('September', 64),
 ('October', 16)]

# Find the average number of products per order, grouped by customer city.


In [7]:
query = """ with count_per_order as
(select orders.order_id, orders.customer_id, count(order_items.order_id) as oc
from orders join order_items
on orders.order_id = order_items.order_id
group by orders.order_id, orders.customer_id)

select customers.customer_city, round(avg(count_per_order.oc),2) average_orders
from customers join count_per_order
on customers.customer_id = count_per_order.customer_id
group by customers.customer_city order by average_orders""" 


cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data,columns = ["customer city","average orders"])
df.head(10)

,customer city,average orders
0,fortim,4.00
1,cairu,4.00
2,epitaciolandia,4.00
3,urucania,4.00
4,brotas de macaubas,4.00
5,tunapolis,4.00
6,porto calvo,4.00
7,ipupiara,4.00
8,capela do alto alegre,4.00
9,americo de campos,4.00


# Calculate the percentage of total revenue contributed by each product category.

In [12]:
query = """ select products.productcategory category,
round((sum(payments.payment_value)/(select sum(payment_value) from payments))*100,2) sales_percentage
from products join order_items
on products.product_id = order_items.product_id
join payments
on payments.order_id = order_items.order_id
group by category order by sales_percentage desc """


cur.execute(query)

data = cur.fetchall()
df = pd.DataFrame(data,columns = ["category","percentage distribution"])
df.head(10)


,category,percentage distribution
0,bed table bath,32.09
1,HEALTH BEAUTY,31.06
2,computer accessories,29.71
3,Furniture Decoration,26.80
4,Watches present,26.78
5,sport leisure,26.09
6,housewares,20.52
7,automotive,15.97
8,Garden tools,15.71
9,Cool Stuff,14.61


# Identify the correlation between product price and the number of times a product has been purchased.


In [16]:
query = """ select products.productcategory,
count(order_items.product_id),
round(avg(order_items.price),2)
from products join order_items
on products.product_id = order_items.product_id
group by products.productcategory """


cur.execute(query)

data = cur.fetchall()
df = pd.DataFrame(data,columns = ["category","order_count","price"])
df.head(10)


,category,order_count,price
0,HEALTH BEAUTY,29010,130.16
1,sport leisure,25923,114.34
2,Cool Stuff,11388,167.36
3,computer accessories,23481,116.51
4,Watches present,17973,201.14
5,housewares,20892,90.79
6,electronics,8301,57.91
7,None,4809,112.00
8,toys,12351,117.55
9,bed table bath,33345,93.30


# Calculate the total revenue generated by each seller, and rank them by revenue.

In [21]:
query = """ select *, dense_rank() over(order by revenue desc) as rn from
(select order_items.seller_id,sum(payments.payment_value) 
revenue from order_items join payments
on order_items.order_id = payments.order_id
group by  order_items.seller_id) as a """


cur.execute(query)

data = cur.fetchall()
data



[('7c67e1448b00f6e969d365cea6b010ab', 507166.9073021412, 1),
 ('1025f0e2d44d7041d6cf58b6550e0bfa', 308222.0398402214, 2),
 ('4a3ca9315b744ce9f8e9374361493884', 301245.26976528764, 3),
 ('1f50f920176fa81dab994f9023523100', 290253.42012761533, 4),
 ('53243585a1d6dc2643021fd1853d8905', 284903.0804977417, 5),
 ('da8622b14eb17ae2831f4ac5b9dab84a', 272219.31931465864, 6),
 ('4869f7a5dfa277a7dca6462dcf3b52b2', 264166.1209387779, 7),
 ('955fee9216a65b617aa5c0531780ce60', 236322.30050226487, 8),
 ('fa1c13f2614d7b5c4749cbc52fecda94', 206513.22986984253, 9),
 ('7e93a43ef30c4f03f38b393420bc753a', 185134.20970630646, 10),
 ('6560211a19b47992c3666cc44a7e94c0', 179657.74904833734, 11),
 ('7a67c85e85bb2ce8582c35f2203ad736', 169030.80002561212, 12),
 ('25c5c91f63607446a97b143d2d535d31', 160534.7397725582, 13),
 ('a1043bafd471dff536d0c462352beb48', 154356.91018915176, 14),
 ('46dc3b2cc0980fb8ec44634e21d2718e', 148864.3400707245, 15),
 ('b37c4c02bda3161a7546a4e6d222d5b2', 145319.04013061523, 16),
 ('620c